# Test cases

This notebook set up and run the test cases in Table 2 using KPP and $k$-$\epsilon$.
- GLS-C01A: the generic length scale (GLS; [Umlauf and Burchard, 2003](https://doi.org/10.1357/002224003322005087)) model in the $k$-$\epsilon$ formulation with the weak-equilibrium stability function by [Canuto et al., 2001](https://doi.org/10.1175/1520-0485(2001)031%3C1413:OTPIOP%3E2.0.CO;2) (C01A).
- KPP-CVMix: KPP implementation in CVMix ([Large et al., 1994](https://doi.org/10.1029/94RG01872), [Griffies et al., 2015](https://github.com/CVMix/CVMix-description/raw/master/cvmix.pdf)).

![table_forcing](table_forcing.png)

In [ ]:
import sys
import copy
sys.path.append("../gotmtool")
from gotmtool import *

## Create a model
Create a model with environment file `../gotmtool/.gotm_env.yaml`, which is created by `gotm_env_init.py`. 

In [ ]:
m = Model(name='ADHOC', environ='../gotmtool/.gotm_env.yaml')

Take a look at what are defined in the environment file.

In [ ]:
for key in m.environ:
    print('{:>15s}: {}'.format(key, m.environ[key]) )

## Build the model

In [ ]:
%%time
m.build()

## Configuration
Initialize the GOTM configuration

In [ ]:
cfg = m.init_config()

Update the configuration

In [ ]:
# setup
title = 'ADHOC'
depth = 200.0
rho0 = 1000
cp = 4218
alphaT = 2e-4
betaS = 8e-4

cfg['title'] = title
cfg['location']['name'] = 'idealized'
cfg['location']['latitude'] = 0.0
cfg['location']['longitude'] = 0.0
cfg['location']['depth'] = depth
cfg['time']['start'] = '2000-01-01 00:00:00'
cfg['time']['stop']  = '2000-01-05 00:00:00'
cfg['time']['dt']    = 60.0

# output
cfg['output'] = {}
cfg['output']['gotm_out'] = {}
cfg['output']['gotm_out']['use'] = True
cfg['output']['gotm_out']['title'] = title
cfg['output']['gotm_out']['time_unit'] = 'dt'
cfg['output']['gotm_out']['time_step'] = 60
cfg['output']['gotm_out']['variables'] = [{}]
cfg['output']['gotm_out']['variables'][0]['source'] = '*'

# forcing
datadir = m.environ['gotmdir_data']
cfg['temperature']['method'] = 'file'
cfg['salinity']['method'] = 'file'
cfg['surface']['fluxes']['heat']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['method'] = 'constant'
cfg['surface']['fluxes']['ty']['method'] = 'constant'
cfg['surface']['swr']['method'] = 'constant'
cfg['surface']['precip']['method'] = 'constant'
cfg['surface']['fluxes']['tx']['constant_value'] = 0.0
cfg['surface']['fluxes']['ty']['constant_value'] = 0.0
cfg['surface']['swr']['constant_value'] = 0.0

# EOS -- use linear
cfg['eq_state']['form'] = 'linear_custom'
cfg['eq_state']['linear']['T0'] = 20.0
cfg['eq_state']['linear']['S0'] = 35.0
cfg['eq_state']['linear']['dtr0'] = -alphaT*rho0
cfg['eq_state']['linear']['dsr0'] = betaS*rho0
cfg['physical_constants']['rho_0'] = rho0


## Create a list of configurations

In [ ]:
# Forcing                                                                                                                                                               

cases = {
    'C1':    {'Qh':  -50.0, 'Qs':  0.0,    'TStag': 'C'},
    'C2':    {'Qh': -100.0, 'Qs':  0.0,    'TStag': 'C'},
    'C4':    {'Qh': -200.0, 'Qs':  0.0,    'TStag': 'C'},
    'C16':   {'Qh': -800.0, 'Qs':  0.0,    'TStag': 'C'},
    'E1':    {'Qh':    0.0, 'Qs': -8.9e-8, 'TStag': 'E'},
    'E4':    {'Qh':    0.0, 'Qs': -3.5e-7, 'TStag': 'E'},
    'S1':    {'Qh': -100.0, 'Qs':  0.0,    'TStag': 'S1'},
    'S10':   {'Qh': -100.0, 'Qs':  0.0,    'TStag': 'S10'},
    'S20':   {'Qh': -100.0, 'Qs':  0.0,    'TStag': 'S20'},
    'T1S0':  {'Qh':  -50.0, 'Qs':  0.0,    'TStag': 'TS'},
    'T1S1':  {'Qh':  -50.0, 'Qs': -8.9e-8, 'TStag': 'TS'},
    'T1S3':  {'Qh':  -50.0, 'Qs': -2.6e-7, 'TStag': 'TS'},
    'T1S15': {'Qh':  -50.0, 'Qs': -1.3e-6, 'TStag': 'TS'},
}

# Resolution
dz = {'1m': 1.0, '10m': 10.0}

# Turbulence methods
turbmethods = [
    'GLS-C01A',
    'KPP-CVMix',
    ]

In [ ]:
for casename in cases:
    print(casename)

In [ ]:
cfgs = []
labels = []

for res in dz:
    print('dz = {:s}'.format(res))
    nlev = int(depth/dz[res])
    cfg['grid']['nlev']  = nlev
    cfg['output']['gotm_out']['k1_stop'] = nlev+1
    cfg['output']['gotm_out']['k_stop'] = nlev
    for casename in cases:
        print(' - {:s}'.format(casename))
        # set surface fluxes
        cfg['surface']['fluxes']['heat']['constant_value'] = cases[casename]['Qh']
        cfg['surface']['precip']['constant_value'] = cases[casename]['Qs']
        cfg['temperature']['file'] = os.path.join(datadir, 'tprof_file_{:s}.dat'.format(cases[casename]['TStag']))
        cfg['salinity']['file'] = os.path.join(datadir, 'sprof_file_{:s}.dat'.format(cases[casename]['TStag']))
        for turbmethod in turbmethods: 
            run_label = '{:s}_{:s}'.format(turbmethod, res)
            labels.append(os.path.join(casename, run_label))
            run_dir = os.path.join(m.environ['gotmdir_run'], m.name, casename, run_label)
            os.makedirs(run_dir, exist_ok=True)
            if turbmethod == 'GLS-C01A':
                cfg['turbulence']['turb_method'] = 'second_order'
                cfg['turbulence']['tke_method'] = 'tke'
                cfg['turbulence']['len_scale_method'] = 'gls'
                cfg['turbulence']['scnd']['method'] =  'weak_eq_kb_eq'
                cfg['turbulence']['scnd']['scnd_coeff'] =  'canuto-a'
                cfg['turbulence']['turb_param']['length_lim'] = 'false'
                cfg['turbulence']['turb_param']['compute_c3'] = 'true'
                cfg['turbulence']['turb_param']['Ri_st'] = 0.25
                cfg['turbulence']['generic']['gen_m'] = 1.5 
                cfg['turbulence']['generic']['gen_n'] = -1.0
                cfg['turbulence']['generic']['gen_p'] = 3.0 
                cfg['turbulence']['generic']['cpsi1'] = 1.44
                cfg['turbulence']['generic']['cpsi2'] = 1.92
                cfg['turbulence']['generic']['cpsi3minus'] = -0.63
                cfg['turbulence']['generic']['cpsi3plus'] = 1.0 
                cfg['turbulence']['generic']['sig_kpsi'] = 1.0 
                cfg['turbulence']['generic']['sig_psi'] = 1.3
            elif turbmethod == 'KPP-CVMix':
                cfg['turbulence']['turb_method'] = 'cvmix'
                cfg['cvmix']['surface_layer']['kpp']['langmuir_method'] = 'none'
            else:
                raise ValueError('Turbulence closure method \'{}\' not defined.'.format(turbmethod))
            cfgs.append(copy.deepcopy(cfg))

## Run the model

In [ ]:
%%time
sims = m.run_batch(configs=cfgs, labels=labels, nproc=4)